In [ ]:
import os.path

import numpy as np
import tensorflow as tf
import src.metrics as nc_metrics
import src.utils as nc_utils
import notebooks.sandbox.models as nc_models

from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import models

2024-02-15 21:13:43.741822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 21:13:43.741887: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 21:13:43.743550: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-15 21:13:43.753561: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-15 21:13:45.483033: W tensorflow/compiler/tf2

In [ ]:
strategy = nc_utils.start_session()

In [3]:
ROOT_DIR = "."
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
MASK_DIR = os.path.join(ROOT_DIR, "masks")

BATCH_SIZE = 16
LR = 1e-4   # Learning rate
EPOCHS = 300

In [4]:
smallest_dimension = nc_utils.get_smallest_image_dimension(IMAGE_DIR)

In [5]:
IMAGE_SIZE = smallest_dimension
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [6]:
print(IMAGE_SIZE)
print(IMAGE_SHAPE)
print(INPUT_SHAPE)

512
(512, 512)
(512, 512, 3)


In [7]:
(x_train_paths, y_train_paths), (x_valid_paths, y_valid_paths), (x_test_paths,
 y_test_paths) = nc_utils.load_data(
    image_directory=IMAGE_DIR,
    mask_directory=MASK_DIR,
    split=0.1
)

image_directory: <class 'str'>
mask_directory: <class 'str'>
image_paths: <class 'list'>
mask_paths: <class 'list'>
image_paths[0]: <class 'str'>
mask_paths[0]: <class 'str'>


In [8]:
print(x_train_paths)

['./images/tumor (159).png', './images/tumor (250).png', './images/tumor (304).png', './images/tumor (53).png', './images/tumor (115).png', './images/tumor (170).png', './images/tumor (42).png', './images/tumor (41).png', './images/tumor (89).png', './images/tumor (290).png', './images/tumor (112).png', './images/tumor (315).png', './images/tumor (32).png', './images/tumor (84).png', './images/tumor (296).png', './images/tumor (226).png', './images/tumor (204).png', './images/tumor (281).png', './images/tumor (195).png', './images/tumor (19).png', './images/tumor (326).png', './images/tumor (154).png', './images/tumor (269).png', './images/tumor (63).png', './images/tumor (111).png', './images/tumor (156).png', './images/tumor (72).png', './images/tumor (142).png', './images/tumor (48).png', './images/tumor (324).png', './images/tumor (169).png', './images/tumor (243).png', './images/tumor (228).png', './images/tumor (275).png', './images/tumor (147).png', './images/tumor (26).png', '.

In [9]:
print(y_train_paths)

['./masks/tumor (159)_mask.png', './masks/tumor (250)_mask.png', './masks/tumor (304)_mask.png', './masks/tumor (53)_mask.png', './masks/tumor (115)_mask.png', './masks/tumor (170)_mask.png', './masks/tumor (42)_mask.png', './masks/tumor (41)_mask.png', './masks/tumor (89)_mask.png', './masks/tumor (290)_mask.png', './masks/tumor (112)_mask.png', './masks/tumor (315)_mask.png', './masks/tumor (32)_mask.png', './masks/tumor (84)_mask.png', './masks/tumor (296)_mask.png', './masks/tumor (226)_mask.png', './masks/tumor (204)_mask.png', './masks/tumor (281)_mask.png', './masks/tumor (195)_mask.png', './masks/tumor (19)_mask.png', './masks/tumor (326)_mask.png', './masks/tumor (154)_mask.png', './masks/tumor (269)_mask.png', './masks/tumor (63)_mask.png', './masks/tumor (111)_mask.png', './masks/tumor (156)_mask.png', './masks/tumor (72)_mask.png', './masks/tumor (142)_mask.png', './masks/tumor (48)_mask.png', './masks/tumor (324)_mask.png', './masks/tumor (169)_mask.png', './masks/tumor (2

In [10]:
train_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_train_paths, y_train_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

In [11]:
validation_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_valid_paths, y_valid_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

In [12]:
test_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_test_paths, y_test_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# Model Creation

In [13]:
data_aug = nc_utils.get_data_augmentation_pipeline()

In [14]:
# Initiating model on GPU
with strategy.scope():
    model = nc_models.unet_model(input_shape=INPUT_SHAPE, augment_data=True)

In [15]:
# Compiling model
model.compile(optimizer=tf.keras.optimizers.Nadam(LR),
              loss='binary_crossentropy',
              metrics=[nc_metrics.dice_coefficient, nc_metrics.continuous_dice_coefficient])

In [16]:
early_stopping = callbacks.EarlyStopping(monitor = 'val_loss',
                                         patience = 5, mode = 'max',
                                         restore_best_weights = True)

# Model Checkpoint

In [17]:
train_steps = len(x_train_paths) // BATCH_SIZE
valid_steps = len(x_valid_paths) // BATCH_SIZE

if len(x_train_paths) % BATCH_SIZE != 0:
    train_steps += 1
if len(x_valid_paths) % BATCH_SIZE != 0:
    valid_steps += 1
    

In [18]:
try:
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=validation_dataset,
        callbacks=[early_stopping],
        steps_per_epoch=train_steps,
        validation_steps=valid_steps)
except Exception as e:
    print("An error occurred:", e)

Epoch 1/300


2024-02-13 18:21:48.388926: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


image_path <class 'str'>
raw_image_array: <class 'numpy.ndarray'>
resized_image_array <class 'numpy.ndarray'>
image_path <class 'str'>
raw_image_array: <class 'numpy.ndarray'>
resized_image_array <class 'numpy.ndarray'>
image_path <class 'str'>
raw_image_array: <class 'numpy.ndarray'>
resized_image_array <class 'numpy.ndarray'>
An error occurred: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
0-th value returned by pyfunc_0 is double, but expects float
	 [[{{node PyFunc}}]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[IteratorGetNext]] [Op:__inference_train_function_11650]
image_path <class 'str'>
raw_image_array: <class 'numpy.ndarray'>
resized_image_array <class 'numpy.ndarray'>
